In [1]:
!python -V

Python 3.9.19


In [3]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [4]:
import mlflow 
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [5]:
mlflow.set_experiment("salary-prediction-experiment")

<Experiment: artifact_location='/Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/02_mlflow/mlruns/1', creation_time=1724071254963, experiment_id='1', last_update_time=1724071254963, lifecycle_stage='active', name='salary-prediction-experiment', tags={}>

In [8]:
train_file_path = '/Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/data/train_file.csv'
val_file_path = '/Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/data/val_file.csv'
test_file_path = '/Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/data/test_file.csv'

In [9]:
train_df = pd.read_csv(train_file_path)
val_df = pd.read_csv(val_file_path)
test_df = pd.read_csv(test_file_path)

In [10]:
train_df.shape, val_df.shape, test_df.shape

((4016, 6), (1339, 6), (1339, 6))

In [11]:
categorical = ['Education', 'JobTitle']
numerical = ['Age','YearsOfExp']

In [12]:
dv = DictVectorizer()

In [13]:
train_dicts = train_df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = val_df[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'Salary'
y_train = train_df[target].values
y_val = val_df[target].values


In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/usr/local/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


18522.687312864502

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv , lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "rashmi")
    mlflow.log_param("train-data-path", "data/train_file.csv")
    mlflow.log_param("validation-data-path", "data/val_file/csv")

    # alpha = 0.01
    # mlflow.log_param("alpha", alpha)
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared= False)
    mlflow.log_metric("rmse", rmse)

/Users/rashmi/anaconda3/envs/exp-track/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "rashmi")
    mlflow.log_param("train-data-path", "data/train_file.csv")
    mlflow.log_param("validation-data-path", "data/val_file/csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lss = Lasso(alpha)
    lss.fit(X_train, y_train)

    y_pred = lss.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared= False)
    mlflow.log_metric("rmse", rmse)

/usr/local/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 511524652228.87286, tolerance: 1113348375.136847
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [5]:
import xgboost as xgb

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/02_mlflow/exp-track-noconda/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <04125532-9495-3051-97BC-F23BE76BA2F9> /Users/rashmi/Documents/rkamath/github/mlopszoomcamp/project/mlops-ds-salary-prediction/02_mlflow/exp-track-noconda/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libo

In [3]:
# Check if DMatrix is available
try:
    print(xgb.DMatrix)
    print("XGBoost is installed correctly.")
except AttributeError:
    print("XGBoost is not installed correctly.")

NameError: name 'xgb' is not defined

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

AttributeError: module 'xgboost' has no attribute 'DMatrix'

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100,1)),
    'learning_rate': hp.loguniform('learning_rate', -3,0), 
    'reg_alpha': hp.loguniform('reg_alpha',-5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6,-1),
    'min_child_weight': hp.loguniform('min_child_weight',-1,3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest, 
    max_evals = 50,
    trials = Trials()
)


NameError: name 'objective' is not defined